# Training ML Model with AI Core Workflow Templates

1. [Authentication](#Authentication)
1. [Scenarios & Executables](#Scenarios-&-Executables)
1. [Artifacts](#Artifacts)
1. [Configurations](#Configurations)
1. [Triggering of Training Pipelines](#Triggering-of-Training-Pipelines)
1. [Serving of Pipelines](# Serving-of-Pipelines)

In [51]:
import requests
import json

## Authentication

Load AI Core service instance credentials. \
The service key of your AI Core instance is generated from the SCP Cockpit.

See: https://help.sap.com/viewer/3c2c5791726047b18597e30ce2f5e109/CLOUD/en-US/c190869ce93b43ec80d576b83810e8f7.html

In [52]:
# Save the service key under the `config/service-key` folder
with open('./config/service-key/enablement-sk.json') as service_key_file:
    service_key = json.load(service_key_file)

base_url = service_key['serviceurls']['ML_API_URL'] + '/v2'

In [53]:
base_url

'https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2'

In [54]:
# Fetch fresh JWT token

response = requests.request("GET", 
                            f"{service_key['url']}/oauth/token?grant_type=client_credentials", 
                            auth=requests.auth.HTTPBasicAuth(service_key['clientid'], service_key['clientsecret']))
token = response.json()["access_token"]
headers = {
  'AI-Resource-Group': 'default',
  'Authorization': f'Bearer {token}'
}

In [55]:
# token

## Scenarios & Executables

Scenario is an umbrella entity that enables grouping design time entities of an ML use case. In the context of a customer tenant (“resource group”), it also encapsulates all information on the runtime state.

Executable is a resource of AI API that can be scheduled for execution after providing a configuration.

See: 
* [Scenario API Definition](https://help.sap.com/doc/4a82ef58d91848fcb57206b334fa0e0a/CLOUD/en-US/ai-core-spec.html#/scenario)
* [Executable API Definition](https://help.sap.com/doc/4a82ef58d91848fcb57206b334fa0e0a/CLOUD/en-US/ai-core-spec.html#/executable)

In [56]:
# List scenarios

url = f"{base_url}/lm/scenarios"

response = requests.request("GET", url, headers=headers)

print(response.text)


{
  "count": 27,
  "resources": [
    {
      "createdAt": "2021-01-08T05:11:02+00:00",
      "description": "glim training scenario",
      "id": "ae0bd260-41ef-4162-81b0-861bd78a8004",
      "modifiedAt": "2021-01-08T05:11:02+00:00",
      "name": "glim-training"
    },
    {
      "createdAt": "2021-01-08T14:24:13+00:00",
      "description": "TensorFlow model serving",
      "id": "aicore-sample-pipeline",
      "modifiedAt": "2021-01-29T16:35:05+00:00",
      "name": "TensorFlow model serving sample scenario"
    },
    {
      "createdAt": "2021-01-19T10:44:22+00:00",
      "description": "test training scenario 1",
      "id": "0de0f069-261a-494f-913f-38c45c25f167",
      "modifiedAt": "2021-01-19T10:44:22+00:00",
      "name": "test-training"
    },
    {
      "createdAt": "2021-01-26T11:40:18+00:00",
      "description": "Amitava Test Mnist",
      "id": "amitava_2",
      "modifiedAt": "2021-01-26T11:40:18+00:00",
      "name": "testscenario"
    },
    {
      "createdAt": 

In [72]:
# List scenario executables 

scenario_id = 'spacyner2202'
url = f"{base_url}/lm/scenarios/{scenario_id}/executables"

response = requests.request("GET", url, headers=headers)

print(response.text)

{
  "count": 1,
  "resources": [
    {
      "createdAt": "2021-02-22T21:49:48+00:00",
      "deployable": false,
      "description": "Spacy Ner",
      "id": "spacyner2202",
      "inputArtifacts": [
        {
          "name": "training-data"
        },
        {
          "name": "training-data"
        }
      ],
      "modifiedAt": "2021-02-22T21:49:48+00:00",
      "name": "spacyner2202",
      "outputArtifacts": [
        {
          "name": "model"
        }
      ],
      "parameters": [
        {
          "name": "training-epochs",
          "type": "string"
        }
      ],
      "scenarioId": "spacyner2202",
      "versionId": "0.1.0"
    }
  ]
}



## Artifacts

Datasets and models are registered as artifacts in AI Core. The actual storage of the artifacts is in remote storage, e.g. AWS S3.

AI Core performs automatic fetching and saving of artifact files from/to the remote storage (AWS S3 for now only).

See: [Artifacts API Definition](https://help.sap.com/doc/4a82ef58d91848fcb57206b334fa0e0a/CLOUD/en-US/ai-core-spec.html#/artifact)

In [64]:
# List dataset in object store

bucket="hcp-8da2bd2a-eb08-4998-baf0-3ceae565e40d"
prefix="spacy"

!aws s3 ls s3://{bucket}/{prefix} --profile aif --recursive
    
# Note: `--profile demo-enablement` is a pre-configured AWS CLI context that enables access to the S3 bucket
# Use your own S3 bucket and access credentials to list, upload, or download files 

2021-06-15 10:25:01       3216 spacy/
2021-04-12 23:31:03       1268 spacy/TRAIN_DATA.json
2021-02-23 00:17:01       1310 spacy/TRAIN_DATA.txt
2021-06-04 16:30:40          0 spacy/abcd.txt
2021-06-02 07:05:23          0 spacy/argo.yaml
2021-05-28 12:21:28       3216 spacy/argoCrud.yaml
2021-04-20 21:27:51       1333 spacy/basic_data.json
2021-02-26 07:31:58       1341 spacy/data/TRAIN_DATA.json
2021-02-22 23:34:43       1310 spacy/data/TRAIN_DATA.txt
2021-03-02 22:46:04      54325 spacy/data/q.png
2021-05-28 12:29:28       3216 spacy/file.json
2021-03-02 22:45:53      54325 spacy/q.png
2021-02-27 11:53:49        506 spacyner2202/0001/model/meta.json
2021-02-27 11:53:49          4 spacyner2202/0001/model/model
2021-02-27 11:53:49        354 spacyner2202/0001/model/ner/cfg
2021-02-27 11:53:49    4010004 spacyner2202/0001/model/ner/model
2021-02-27 11:53:49        486 spacyner2202/0001/model/ner/moves
2021-02-27 11:53:49      88955 spacyner2202/0001/model/tokenizer
2021-02-27 11:53:49    

In [73]:
# Register dataset as artifact in AI Core

url = f"{base_url}/lm/artifacts"

payload = { 
  'name': 'spacyner2202',
  'kind': 'dataset',
  'url': 'ai://default/spacy/',
  'description': 'spacyner2202',
  'scenarioId': 'spacyner2202'
}

response = requests.request("POST", url, headers=headers, json=payload)

print(response.text)

artifact_id = json.loads(response.text)['id']

{
  "id": "e4b2f6e1-e551-4d17-a780-7ecc1120e202",
  "message": "Artifact acknowledged",
  "url": "ai://default/spacy/"
}



In [74]:
url = f"{base_url}/lm/artifacts"

response = requests.request("GET", url, headers=headers)

# print(response.text)

In [75]:
scenario_id ='spacyner2202'

## Configurations

Configurations are grouping definition of input artifacts and parameters that are be used to parametrize an executable.

See: [Configuration API Definition](https://help.sap.com/doc/4a82ef58d91848fcb57206b334fa0e0a/CLOUD/en-US/ai-core-spec.html#/configuration)

In [76]:
# Create configuration for executable

url = f"{base_url}/lm/configurations"

conf_payload = {
  "name": "spacyner2202",
  "executableId": "spacyner2202",
  "scenarioId": f"{scenario_id}",
  "parameterBindings": [
    {
      "key": "training-epochs",
      "value": "100"
    }
  ],
  "inputArtifactBindings": [
    {
      "key": "training-data",
      "artifactId": f"{artifact_id}"
    }
  ]
}

response = requests.request("POST", url, headers=headers, json=conf_payload)
print(response.text)
conf_id = response.json()['id']

{
  "id": "4464b254-e366-4dba-8040-57da99b4c8f3",
  "message": "Configuration created"
}



In [77]:
# Show configurations
url = f"{base_url}/lm/configurations?scenarioId=spacyner2202"

response = requests.request("GET", url, headers=headers)

# print(response.text)

In [78]:
# Show configuration
# conf_id = '65521919-cbbc-4701-ae79-91f584b604d7'
url = f"{base_url}/lm/configurations/{conf_id}"

response = requests.request("GET", url, headers=headers)

print(response.text)

{
  "createdAt": "2021-06-17T21:25:22Z",
  "executableId": "spacyner2202",
  "id": "4464b254-e366-4dba-8040-57da99b4c8f3",
  "inputArtifactBindings": [
    {
      "artifactId": "e4b2f6e1-e551-4d17-a780-7ecc1120e202",
      "key": "training-data"
    }
  ],
  "name": "spacyner2202",
  "parameterBindings": [
    {
      "key": "training-epochs",
      "value": "100"
    }
  ],
  "scenarioId": "spacyner2202"
}



In [79]:
conf_id

'4464b254-e366-4dba-8040-57da99b4c8f3'

## Triggering of Training Pipelines

Execution is an instance of running a training pipeline, i.e. an executable.

See: [Execution API Definition](https://help.sap.com/doc/4a82ef58d91848fcb57206b334fa0e0a/CLOUD/en-US/ai-core-spec.html#/execution)

In [80]:
# Start execution of executable with given config

url = f"{base_url}/lm/configurations/{conf_id}/executions"

response = requests.request("POST", url, headers=headers)

print(response.text)

exec_id = response.json()["id"]

{
  "id": "ec7df6a70804fd56",
  "message": "Execution scheduled",
  "status": "UNKNOWN",
  "targetStatus": "COMPLETED"
}



In [96]:
# Check status of executable
# exec_id = "ea202ef744b43b37"
url = f"{base_url}/lm/executions/{exec_id}"

response = requests.request("GET", url, headers=headers)

print(response.text)

{
  "configurationId": "4464b254-e366-4dba-8040-57da99b4c8f3",
  "configurationName": "spacyner2202",
  "createdAt": "2021-06-17T21:25:35Z",
  "executableId": "spacyner2202",
  "id": "ec7df6a70804fd56",
  "modifiedAt": "2021-06-17T21:25:47Z",
  "outputArtifacts": [],
  "scenarioId": "spacyner2202",
  "status": "DEAD",
  "statusDetails": {
    "details": [
      {
        "container_name": "main",
        "exit_code": 1,
        "last_log_messages": "t):  File \"/***\", line 125, in <module>    train()  File \"/***\", line 33, in train    with open(***(folder,'***') ,encoding=\"utf8\") as fp:NotADirectoryError: [Errno 20] Not a directory: ***/***'...",
        "message": "",
        "pod_name": "ec7df6a70804fd56-916270895",
        "ready": false
      },
      {
        "container_name": "wait",
        "exit_code": 0,
        "last_log_messages": "tainers [7c0e2c110a8eb5973fcba3f199f096e6d504bc64a7d89450ed67f12a662b767a] killed successfully\"time=\"2021-06-17T21:26:***\" level=info ms

In [48]:
#Output Artifact Id

outputArtifactID = response.json()['outputArtifacts'][0]['id']
outputArtifactID

IndexError: list index out of range

## Serving of  Pipelines

Execution is an instance of running a training pipeline, i.e. an executable.

In [53]:
!aws s3 ls s3://{bucket}/{exec_id}/ --profile aif --recursive --human-readable --summarize

2021-04-20 22:35:30  517 Bytes ea202ef744b43b37/model/meta.json
2021-04-20 22:35:30    4 Bytes ea202ef744b43b37/model/model
2021-04-20 22:35:30  354 Bytes ea202ef744b43b37/model/ner/cfg
2021-04-20 22:35:30    3.8 MiB ea202ef744b43b37/model/ner/model
2021-04-20 22:35:30  502 Bytes ea202ef744b43b37/model/ner/moves
2021-04-20 22:35:30   86.9 KiB ea202ef744b43b37/model/tokenizer
2021-04-20 22:35:30     1 Byte ea202ef744b43b37/model/vocab/key2row
2021-04-20 22:35:30   28.7 MiB ea202ef744b43b37/model/vocab/lookups.bin
2021-04-20 22:35:30   12.0 KiB ea202ef744b43b37/model/vocab/strings.json
2021-04-20 22:35:30  128 Bytes ea202ef744b43b37/model/vocab/vectors

Total Objects: 10
   Total Size: 32.6 MiB


In [57]:
import os
os.getcwd()

'C:\\Users\\I304524\\Desktop\\DaysOfCodeRound2\\3\\spacyServe'

In [60]:
!aws s3 cp s3://{bucket}/{exec_id}/   "C:\\Users\\I304524\\Desktop\\DaysOfCodeRound2\\3\\spacyServe\\docker-serving\\model"   --profile aif --recursive

Completed 354 Bytes/32.6 MiB (1.1 KiB/s) with 10 file(s) remaining
download: s3://hcp-8da2bd2a-eb08-4998-baf0-3ceae565e40d/ea202ef744b43b37/model/ner/cfg to docker-serving\model\model\ner\cfg
Completed 354 Bytes/32.6 MiB (1.1 KiB/s) with 9 file(s) remaining
Completed 358 Bytes/32.6 MiB (1.0 KiB/s) with 9 file(s) remaining
download: s3://hcp-8da2bd2a-eb08-4998-baf0-3ceae565e40d/ea202ef744b43b37/model/model to docker-serving\model\model\model
Completed 358 Bytes/32.6 MiB (1.0 KiB/s) with 8 file(s) remaining
Completed 12.4 KiB/32.6 MiB (33.8 KiB/s) with 8 file(s) remaining
Completed 12.9 KiB/32.6 MiB (34.9 KiB/s) with 8 file(s) remaining
Completed 13.0 KiB/32.6 MiB (34.8 KiB/s) with 8 file(s) remaining
Completed 13.0 KiB/32.6 MiB (34.4 KiB/s) with 8 file(s) remaining
download: s3://hcp-8da2bd2a-eb08-4998-baf0-3ceae565e40d/ea202ef744b43b37/model/meta.json to docker-serving\model\model\meta.json
Completed 13.0 KiB/32.6 MiB (34.4 KiB/s) with 7 file(s) remaining
download: s3://hcp-8da2bd2a-eb

In [40]:
# Serving Scenario 

scenario_id = 'materialbasicdataserve'

executable_id = "materialbasicdataserve"


In [41]:
# Create configuration for executable

url = f"{base_url}/lm/configurations"

conf_payload = {
    "name": "materialbasicdataserve",
    "executableId": f"{executable_id}",
    "versionId": "0.1.0",
    "scenarioId": f"{scenario_id}",
    "parameterBindings": [
        {
            "key": "model",
            "value": "model"
        }
    ],
    "inputArtifactBindings": [
        {
            "key": "model",
            "artifactId": f"{outputArtifactID}"
        }
    ]
}

response = requests.request("POST", url, headers=headers, json=conf_payload)
print(response.text)
conf_id = response.json()['id']

{
  "id": "87c4fcd8-d050-4bbe-bd52-923b0befd38e",
  "message": "Configuration created"
}



In [42]:
# Show configuration
url = f"{base_url}/lm/configurations/{conf_id}"

response = requests.request("GET", url, headers=headers)

print(response.text)

{
  "createdAt": "2021-05-12T10:39:40Z",
  "executableId": "materialbasicdataserve",
  "id": "87c4fcd8-d050-4bbe-bd52-923b0befd38e",
  "inputArtifactBindings": [
    {
      "artifactId": "3eaa38b4-c6d7-43a4-8c56-10fb27fdb502",
      "key": "model"
    }
  ],
  "name": "materialbasicdataserve",
  "parameterBindings": [
    {
      "key": "model",
      "value": "model"
    }
  ],
  "scenarioId": "materialbasicdataserve"
}



In [43]:
# # # # Start deployment of model serving template with given config

# url = f"{base_url}/lm/configurations/{conf_id}/deployments"

# response = requests.request("POST", url, headers=headers)

# print(response.text)

# deployment_id = response.json()["id"]

{
  "deploymentUrl": "",
  "id": "d684ab99b148a2ef",
  "message": "Deployment scheduled.",
  "status": "UNKNOWN"
}



In [44]:
# Spacy Running Deployment - Lets not create another 

deployment_id ="d684ab99b148a2ef"

In [54]:
# Check status of deployment

url = f"{base_url}/lm/deployments/{deployment_id}"

response = requests.request("GET", url, headers=headers)

print(response.text)

{
  "configurationId": "87c4fcd8-d050-4bbe-bd52-923b0befd38e",
  "configurationName": "materialbasicdataserve",
  "createdAt": "2021-05-12T10:39:52Z",
  "deploymentUrl": "https://d684ab99b148a2ef-rg-aeb65ab8-cd927cab.serving.internalprod.eu-central-1.aws.ml.hana.ondemand.com",
  "id": "d684ab99b148a2ef",
  "modifiedAt": "2021-05-12T10:40:55Z",
  "scenarioId": "materialbasicdataserve",
  "status": "RUNNING",
  "targetStatus": "RUNNING"
}



In [97]:
deployment_url = response.json()['deploymentUrl']

In [98]:
deployment_url

'https://d5516731e5e84d92-rg-aeb65ab8-cd927cab.serving.internalprod.eu-central-1.aws.ml.hana.ondemand.com'

In [99]:
url

'https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/lm/deployments/d5516731e5e84d92'

In [125]:
url = f"{deployment_url}/v1/models/model:predict"
with open('input.txt', 'r') as input_file:
    text = input_file.read()
inputs = {
"text": text,
}
print(text)
print('************************************Entities***************************************')
response = requests.request("POST", url, headers=headers, json = inputs)                          
print(response.text)
print('************************************Entities***************************************')


Intel Lenovo Laptop is a good laptop.The net and gross weights are 1.1 and 1.3 KG respectively. The length is  120 cm. The width is 60 cm. The HOEHE of the product is 15 cm. The EAN codes is 56454454354.
************************************Entities***************************************
[{"MATKX":"Intel Lenovo Laptop"},{"BRGEW":"1.3"},{"GEWEI":"KG"},{"BRGEW":"120"},{"MEABM":"cm"},{"BREIT":"60"},{"MEABM":"cm"},{"MEABM":"cm"},{"EAN11":"56454454354"}]
************************************Entities***************************************


In [72]:
import speech_recognition as sr

r = sr.Recognizer()

with sr.Microphone() as source:
    print('******Speak*************')
    audio = r.listen(source)
    
    try:
        text = r.recognize_google(audio);
        print('**********Spoken words******')
        print(text)
    except:
        print('********* I had trouble catching that')



******Speak*************
**********Spoken words******
it weighs 150kg in net and 120kg with packaging


In [92]:
url = f"{deployment_url}/v1/models/model:predict"
inputs = {
"text": text,
}
print(text)
print('************************************Entities***************************************')
response = requests.request("POST", url, headers=headers, json = inputs)                          
print(response.text)
print('************************************Entities***************************************')


To Mitra, Amitava;\n\nSubject New product Information\nDear Customer ,
        \n\nThanks for your enquiry. The new product is of length 1000cm , 
        width 645 cm and height of 436 cm. It weighs 115 KG in Net and 120 KG with packaging.
        The EAN code is 55324324324387. The lead time is around 20 days from confirmation 
        of a firm order.\n\nKind Regards\n\nAmitava\n\nAmitava Mitra
************************************Entities***************************************
[{"MEABM":"cm"},{"BRGEW":"115"},{"GEWEI":"KG"},{"BRGEW":"120"},{"GEWEI":"KG"},{"EAN11":"55324324324387"}]
************************************Entities***************************************


In [44]:
text[3]

'{"BRGEW":"120"}'

In [20]:
# Check status of deployments

url = f'{base_url}/lm/deployments'

response = requests.request('GET', url, headers=headers)

print(response.text)

{
  "count": 0,
  "resources": []
}



In [12]:
# stop all deployments

url = f'{base_url}/lm/deployments'

response = requests.request('GET', url, headers=headers)

deployments = response.json()['resources']

for depl in deployments:
    depl_url = f'{base_url}/lm/deployments/{depl["id"]}'
    payload = {'targetStatus': 'STOPPED'}
    response = requests.request('PATCH', depl_url, headers=headers, json=payload)
    print(f'Stopping deployment: {depl["id"]}')
    print(response.text)

Stopping deployment: dff5cbdb0a1675bb
{
  "id": "dff5cbdb0a1675bb",
  "message": "Deployment modification scheduled"
}

Stopping deployment: d5516731e5e84d92
{
  "id": "d5516731e5e84d92",
  "message": "Deployment modification scheduled"
}

Stopping deployment: d54e8ecec5052a8e
{
  "id": "d54e8ecec5052a8e",
  "message": "Deployment modification scheduled"
}

Stopping deployment: dd8ff96753bc513b
{
  "id": "dd8ff96753bc513b",
  "message": "Deployment modification scheduled"
}

Stopping deployment: d04fb26afb2289cd
{
  "id": "d04fb26afb2289cd",
  "message": "Deployment modification scheduled"
}

Stopping deployment: d07edf8b4c2db243
{
  "id": "d07edf8b4c2db243",
  "message": "Deployment modification scheduled"
}

Stopping deployment: d3890d953752c236
{
  "id": "d3890d953752c236",
  "message": "Deployment modification scheduled"
}

Stopping deployment: de689a02134a90a8
{
  "id": "de689a02134a90a8",
  "message": "Deployment modification scheduled"
}

Stopping deployment: d155667532375a51
{


In [19]:
# after the deployments move to 'STOPPED' state, you can delete them

url = f'{base_url}/lm/deployments'

response = requests.request('GET', url, headers=headers)

deployments = response.json()['resources']

for depl in deployments:
    depl_url = f"{base_url}/lm/deployments/{depl['id']}"
    response = requests.request('DELETE', depl_url, headers=headers)
    print(f'Stopping deployment: {depl["id"]}')
    print(response.text)

Stopping deployment: dff5cbdb0a1675bb
{
  "id": "dff5cbdb0a1675bb",
  "message": "Deletion scheduled",
  "targetStatus": "DELETED"
}

Stopping deployment: d5516731e5e84d92
{
  "id": "d5516731e5e84d92",
  "message": "Deletion scheduled",
  "targetStatus": "DELETED"
}

Stopping deployment: d54e8ecec5052a8e
{
  "id": "d54e8ecec5052a8e",
  "message": "Deletion scheduled",
  "targetStatus": "DELETED"
}

Stopping deployment: dd8ff96753bc513b
{
  "id": "dd8ff96753bc513b",
  "message": "Deletion scheduled",
  "targetStatus": "DELETED"
}

Stopping deployment: d04fb26afb2289cd
{
  "id": "d04fb26afb2289cd",
  "message": "Deletion scheduled",
  "targetStatus": "DELETED"
}

Stopping deployment: d07edf8b4c2db243
{
  "id": "d07edf8b4c2db243",
  "message": "Deletion scheduled",
  "targetStatus": "DELETED"
}

Stopping deployment: d3890d953752c236
{
  "id": "d3890d953752c236",
  "message": "Deletion scheduled",
  "targetStatus": "DELETED"
}

Stopping deployment: de689a02134a90a8
{
  "id": "de689a02134a9